In [ ]:
import pandas as pd

from sklearn.preprocessing import LabelEncoder, PolynomialFeatures

In [ ]:
label_encoder = LabelEncoder()
poly = PolynomialFeatures(degree=2, interaction_only=True, include_bias=False)

In [ ]:
raw_df = pd.read_csv(r"../data/flight_delay.csv")
raw_df

In [ ]:
missing_values = raw_df.isna().sum()
print("Missing values per columns:\n", missing_values)

In [ ]:
raw_df = raw_df.dropna(subset=["Org_Airport", "Dest_Airport"])

In [ ]:
raw_df["DepDate"] = pd.to_datetime(raw_df["Date"], dayfirst=True)
raw_df = raw_df.drop(columns=["Date"])

In [ ]:
def process_time_column(df, col_name, date_col):
    df[col_name] = df[col_name].apply(lambda x: str(x).zfill(4))
    df[col_name] = df[date_col].astype(str) + " " + df[col_name]
    df[col_name] = pd.to_datetime(df[col_name], format="%Y-%m-%d %H%M", errors="coerce")
    return df[col_name]

raw_df["DepTime"] = process_time_column(raw_df, "DepTime", "DepDate")
raw_df["ArrTime"] = raw_df.apply(lambda x: x["DepTime"] + pd.offsets.Minute(x["ActualElapsedTime"]), axis=1)
raw_df["ArrDate"] = raw_df["ArrTime"].dt.date
raw_df["CRSArrTime"] = process_time_column(raw_df, "CRSArrTime", "ArrDate")
raw_df["CRSDepTime"] = raw_df.apply(lambda x: x["CRSArrTime"] - pd.offsets.Minute(x["CRSElapsedTime"]), axis=1)

In [ ]:
raw_df["DepHour"] = raw_df["DepTime"].dt.hour
raw_df["DepTimeMinutes"] = raw_df["DepTime"].dt.hour * 60 + raw_df["DepTime"].dt.minute
raw_df["CRSDepHour"] = raw_df["CRSDepTime"].dt.hour
raw_df["CRSDepTimeMinutes"] = raw_df["CRSDepTime"].dt.hour * 60 + raw_df["CRSDepTime"].dt.minute
raw_df["ArrHour"] = raw_df["ArrTime"].dt.hour
raw_df["ArrTimeMinutes"] = raw_df["ArrTime"].dt.hour * 60 + raw_df["ArrTime"].dt.minute
raw_df["CRSArrHour"] = raw_df["CRSArrTime"].dt.hour
raw_df["CRSArrTimeMinutes"] = raw_df["CRSArrTime"].dt.hour * 60 + raw_df["CRSArrTime"].dt.minute

In [ ]:
day_map = {1: "MON", 2: "TUE", 3: "WED", 4: "THU", 5: "FRI", 6: "SAT", 7: "SUN"}
raw_df["DayOfWeekEncoded"] = raw_df["DayOfWeek"].replace(to_replace=7, value=0)
raw_df["DayOfWeek"] = raw_df["DayOfWeek"].replace(day_map)

In [ ]:
raw_df["Month"] = raw_df["DepDate"].dt.month

In [ ]:
raw_df["NonCarrierDelay"] = raw_df["WeatherDelay"] + raw_df["NASDelay"] + raw_df["SecurityDelay"] + raw_df["LateAircraftDelay"]
raw_df["FlightID"] = raw_df["UniqueCarrier"] + raw_df["FlightNum"].astype(str)
raw_df["FlightIDEncoded"] = label_encoder.fit_transform(raw_df["FlightID"])

In [ ]:
raw_df["Origin_Dep_Count"] = raw_df.groupby(["Origin", "DepDate", "CRSDepHour"])["FlightNum"].transform("count")
raw_df["Dest_Arr_Count"] = raw_df.groupby(["Dest", "ArrDate", "CRSArrHour"])["FlightNum"].transform("count")

In [ ]:
raw_df = raw_df.dropna(subset=["DepTime", "ArrTime", "DepHour", "ArrHour", "Origin_Dep_Count", "Dest_Arr_Count"])

In [ ]:
def poly_transform(df, columns: list):
    interaction_features = poly.fit_transform(df[columns])
    interaction_df = pd.DataFrame(interaction_features, columns=poly.get_feature_names_out(columns))
    interaction_df.rename(columns={f"{columns[0]} {columns[1]}": f"{columns[0]}_{columns[1]}"}, inplace=True)
    return interaction_df[f"{columns[0]}_{columns[1]}"]

dep_day_df =  poly_transform(raw_df, ["DepTimeMinutes", "DayOfWeekEncoded"])
arr_day_df = poly_transform(raw_df, ["CRSArrTimeMinutes", "DayOfWeekEncoded"])
raw_df = pd.concat([raw_df, dep_day_df, arr_day_df], axis=1)

In [ ]:
raw_df = raw_df.drop_duplicates(inplace=False)

In [ ]:
raw_df = raw_df.dropna(subset=["DepTime", "ArrTime", "DepHour", "ArrHour", "DepTimeMinutes_DayOfWeekEncoded", "CRSArrTimeMinutes_DayOfWeekEncoded", "Origin_Dep_Count", "Dest_Arr_Count"])

In [ ]:
missing_values = raw_df.isna().sum()
print("Missing values per columns:\n", missing_values)

In [ ]:
column_order = ["DayOfWeek", "DayOfWeekEncoded", "Month", "DepDate", "DepTime", "DepTimeMinutes", "DepHour", "CRSDepTime", "CRSDepTimeMinutes", "CRSDepHour", "ArrDate", "ArrTime", "ArrTimeMinutes", "ArrHour", "CRSArrTime", "CRSArrTimeMinutes", "CRSArrHour", "Distance", "ActualElapsedTime", "CRSElapsedTime", "AirTime", "UniqueCarrier", "FlightNum", "FlightID", "FlightIDEncoded", "Airline", "TailNum", "Origin", "Org_Airport", "Dest", "Dest_Airport", "TaxiOut", "TaxiIn", "Cancelled", "CancellationCode", "Diverted", "DepDelay", "ArrDelay", "CarrierDelay", "NonCarrierDelay", "WeatherDelay", "NASDelay", "SecurityDelay", "LateAircraftDelay", "DepTimeMinutes_DayOfWeekEncoded", "CRSArrTimeMinutes_DayOfWeekEncoded", "Origin_Dep_Count", "Dest_Arr_Count"]
ordered_df = raw_df[column_order]

In [ ]:
ordered_df.to_csv(r"../data/cleaned_data.csv", index=False)

In [ ]:
ordered_df